In [1]:
# IMPORTS

# tensorflow
import tensorflow as tf

# keras
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Dropout, concatenate, BatchNormalization
from keras.models import Model

# other
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import albumentations as A

In [4]:
# config

# set pathes 
test_folder = './test_v2/'
train_folder = './train_v2/'

# model hyperparameters
batch_size = 32

## Load CSV

In [7]:
# read and reformat csv
df = pd.read_csv(f'train_ship_segmentations_v2.csv')
df = pd.DataFrame(df.groupby('ImageId')['EncodedPixels']
                          .apply(lambda x: None if type(x.values[0]) == float else ' '.join(x.astype(str))))
df = df.drop('6384c3e78.jpg')  # remove corrupted image
df

,EncodedPixels
ImageId,
00003e153.jpg,None
0001124c7.jpg,None
000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
0001b1832.jpg,None
...,...
fffedbb6b.jpg,None
ffff2aa57.jpg,None
ffff6e525.jpg,None


In [3]:
# limit df
df = df[:10000]

## RLE

In [3]:
# creates a photo from RLE
def RLE_decoder(RLE:str, size=(768, 768)):  
    segmented_photo = np.zeros((size[0] * size[1]))  # 1D array for black image
    if RLE == None:
        return tf.image.transpose(np.reshape(segmented_photo, (size[0], size[1], 1)))
    RLE = [int(x) for x in RLE.split(' ')]  # get RLE indices
    for idx in range(0, len(RLE), 2):  # change segmented pixels
        segmented_photo[ RLE[idx] : RLE[idx]+RLE[idx+1] ] = 1
    segmented_photo = np.transpose(np.reshape(segmented_photo, (size[0], size[1], 1)), (1,0,2))  
    return segmented_photo

# creates a RLE from photo
def RLE_encoder(segmented_photo): 
    # transpose and flatten
    segmented_photo = np.transpose(segmented_photo, (1,0,2))
    segmented_photo = segmented_photo.flatten() > 0.5  # segmented pixels count if its value > 0.5
    
    # calculate RLE
    RLE_count = [0]
    RLE_index = [0]
    last_pixel = False
    for idx in range(len(segmented_photo)):
        pixel = segmented_photo[idx]
        if pixel == last_pixel:  
            RLE_count[-1] += 1
        else:
            RLE_count.append(1)
            RLE_index.append(idx)
            last_pixel = not last_pixel

    # convert RLE to string
    RLE = ''
    for idx in range(1,len(RLE_count), 2):
        if RLE != '':
            RLE += ' '
        RLE += f'{RLE_index[idx]} {RLE_count[idx]}'
    return RLE

## Data loaders

In [4]:
# split dataframe into train and valid
df_valid = df.sample(frac=0.1)
df_train = df.drop(df_valid.index)
print(f'Train set: {len(df_train)} samples')
print(f'Validation set: {len(df_valid)} samples')

# augmentations for both image and mask
augment_both = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.3, scale_limit=0.3, rotate_limit=180, p=1, border_mode=0) 
])
# augmentations only for image
augment_image = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1),
    A.MultiplicativeNoise(multiplier=(0.8, 1.2))
])

# generate pair of X and Y with shuffling
def train_generator():
    shuffled_df = df_train.sample(frac=1)  # create shuffled df
    while True:
        for index, row in shuffled_df.iterrows():
            # load image and mask
            X = np.array(tf.keras.utils.load_img(train_folder+index))  # load image
            Y = np.array(RLE_decoder(row.values[0]))  # get segmentation mask from RLE
            # apply augmentations
            X = augment_image(image=X)['image'] / 255  # normalization after augmentation
            res = augment_both(image=X, mask=Y)
            yield res['image'], res['mask']
        shuffled_df = df_train.sample(frac=1)  # reshuffle df
        
# generate pair of X and Y with shuffling
def valid_generator():
    shuffled_df = df_valid.sample(frac=1)  # create shuffled df
    while True:
        for index, row in shuffled_df.iterrows():
            # load image and mask
            X = np.array(tf.keras.utils.load_img(train_folder+index))  # load image
            Y = np.array(RLE_decoder(row.values[0]))  # get segmentation mask from RLE
            # apply augmentations
            X = augment_image(image=X)['image'] / 255  # normalization after augmentation
            res = augment_both(image=X, mask=Y)
            yield res['image'], res['mask']
        shuffled_df = df_valid.sample(frac=1)  # reshuffle df
        
# create Datasets from custom generators
ds_train = tf.data.Dataset.from_generator(
    train_generator, 
    output_shapes=((768,768,3), (768,768,1)),
    output_types=(tf.float32, tf.float32))
ds_valid = tf.data.Dataset.from_generator(
    valid_generator, 
    output_shapes=((768,768,3), (768,768,1)),
    output_types=(tf.float32, tf.float32))

# optimize datasets
ds_train = ds_train.batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_valid = ds_valid.batch(batch_size).prefetch(tf.data.AUTOTUNE)


Train set: 173300 samples
Validation set: 19256 samples


## UNet set up

In [5]:
# define UNet architecture

def down_block(inputs, features:int, factor:int):
    conv = Conv2D(features, 3, activation='relu', padding='same')(inputs)
    conv = BatchNormalization()(conv)
    conv = Conv2D(features, 3, activation='relu', padding='same')(conv)
    conv = BatchNormalization()(conv)
    return MaxPooling2D(pool_size=factor)(conv), conv

def up_block(inputs, skip_conn, features:int, factor:int):
    up = Conv2DTranspose(features, factor, strides=factor, padding='same')(inputs)
    up = BatchNormalization()(up)
    up = concatenate([up, skip_conn])
    conv = Conv2D(features, 3, activation='relu', padding='same')(up)
    conv = BatchNormalization()(conv)
    conv = Conv2D(features, 3, activation='relu', padding='same')(conv)
    conv = BatchNormalization()(conv)
    return conv

def unet(input_size=(768, 768, 3)):
    inputs = Input(input_size)
    
    # downsampling path
    pool1, conv1 = down_block(inputs, 8, 2)
    pool2, conv2 = down_block(pool1, 16, 2)
    pool3, conv3 = down_block(pool2, 32, 2)
    
    # bottom of U-Net
    conv_bottom = Conv2D(64, 3, activation='relu', padding='same')(pool3)
    conv_bottom = Conv2D(64, 3, activation='relu', padding='same')(conv_bottom)
    drop_bottom = Dropout(0.1)(conv_bottom)
    
    # upsampling path
    conv4 = up_block(drop_bottom, conv3, 32, 2)
    conv5 = up_block(conv4, conv2, 16, 2)
    conv6 = up_block(conv5, conv1, 8, 2)

    # output
    outputs = Conv2D(1, 1, activation='sigmoid', padding='same')(conv6)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [6]:
# create model
model = unet()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 768, 768, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 768, 768, 8)  224         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 768, 768, 8)  32         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [ ]:
# load model
model = tf.keras.models.load_model('model.h5', compile=False)
model.summary()

In [12]:
# prepare model for training

# dice score
def dice_loss(y_true, y_pred, smooth=1e-7):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2.0 * intersection + smooth) / (union + smooth)
    return 1 - dice

# learning rate decay
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=900,
    decay_rate=0.7)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), loss=dice_loss)

## Model training

In [13]:
# set callbacks

# saving best model
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model.h5',
    monitor='val_loss',
    mode='min',
    save_best_only=True)

# early stopping
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# fit the model
history = model.fit(ds_train, 
                    validation_data=ds_valid, 
                    steps_per_epoch=len(df_train) // batch_size, 
                    validation_steps=len(df_valid) // batch_size,
                    epochs=100,
                    callbacks=[checkpoint_callback, early_stop_callback])

In [ ]:
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.show()